In [ ]:
from typing import Tuple
import pandas as pd
import numpy as np
import geopandas as gpd 
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist
from scipy.linalg import inv


plt.style.use('seaborn-v0_8')  
plt.rcParams['font.family'] = 'Arial'  

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from mypackage import dir

# Environment variables
modality = 'u'
project = 'Ciencia de los datos'
data = dir.make_dir_line(modality, project) 
processed = data('processed')
models = data('models')
outputs = data('outputs')

In [ ]:
# Definir columnas numéricas y categóricas
numeric_features = ['value', 'females', 'jóvenes', 'mayores', 'divorced', 'married']
orden_categorias = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
def cargar_datos(table_name: str) -> pd.DataFrame:
    df = pd.read_parquet(processed / f'{table_name}.parquet.gzip')
    print(f'Loaded table: {table_name}')
    return df

# Función para cargar los datos en la base de datos
def cargar_datos_geo(table_name: str) -> gpd.GeoDataFrame:
    geo = gpd.read_file(processed / f'{table_name}.geojson', encoding='utf-8')
    print(f'Loaded geo table: {table_name}')
    return geo

def cargar_en_db(df: pd.DataFrame, table_name: str) -> None:
    df.to_parquet(models/f'{table_name}.parquet.gzip', compression='gzip') 
    print(f'Saved table: {table_name}')
    

def calcular_proporciones(df: pd.DataFrame, code: str) -> pd.DataFrame:
    # Agrupaciones y cálculos iniciales
    proporciones_t = df.groupby(['itter107', 'territory', code], as_index=False)['value'].sum()
    
    # Lista de categorías para calcular proporciones
    categorias = [
        ('gender', ['females', 'males']),
        ('grupo_edad', ['jóvenes', 'adultos', 'mayores']),
        ('marital_status', ['divorced', 'married', 'never married'])
    ]
    
    # Procesamiento dinámico de cada categoría
    dfs = [proporciones_t]
    
    for col, subcats in categorias:
        # Calcular proporciones y pivotar
        temp_df = (
            df.groupby(['territory', col], as_index=False)['value'].sum()
            .pivot_table(index='territory', columns=col, values='value', fill_value=0)
            .reset_index()
        )
        dfs.append(temp_df)
    
    # Combinar todos los DataFrames
    proporciones = dfs[0]
    for df_temp in dfs[1:]:
        proporciones = proporciones.merge(df_temp, on='territory')
    
    # Calcular proporciones dividiendo por el valor total
    columnas_a_normalizar = [col for _, subcats in categorias for col in subcats]
    proporciones[columnas_a_normalizar] = proporciones[columnas_a_normalizar].div(proporciones['value'], axis=0)
    
    # # Seleccionar columnas finales
    # columnas_finales = [
    #     'itter107', 'territory', 'value',
    #     'females', 'jóvenes', 'mayores',
    #     'divorced', 'married'
    # ]
    
    # return proporciones[columnas_finales]
    return proporciones


def optimal_kmeans(X: np.ndarray, max_k: int = 10) -> Tuple[int, KMeans]:
    """
    Encuentra el número óptimo de clusters usando el método de la silueta y gráfica los resultados.
    
    Args:
        X: Datos preprocesados (array numpy de 2D)
        max_k: Número máximo de clusters a probar (por defecto 10)
        
    Returns:
        Tuple con:
            - best_k: Número óptimo de clusters encontrado
            - best_model: Modelo KMeans entrenado con el k óptimo
    
    Ejemplo:
        >>> best_k, best_model = optimal_kmeans(X_scaled)
    """
    # Validación de parámetros
    if max_k < 2:
        raise ValueError("max_k debe ser al menos 2")
    
    # Almacenamiento de métricas
    metrics = {
        'k_values': [],
        'silhouette_scores': [],
        'inertias': []
    }
    
    best_k, best_score, best_model = 2, -1, None
    
    for k in range(2, max_k + 1):
        model = KMeans(n_clusters=k, random_state=42, n_init='auto').fit(X)
        score = silhouette_score(X, model.labels_)
        
        # Actualización de métricas
        metrics['k_values'].append(k)
        metrics['silhouette_scores'].append(score)
        metrics['inertias'].append(model.inertia_)
        
    # Actualización del mejor modelo
    elbow_k = np.argmin(np.diff(metrics['inertias'], 2)) + 2
    # guarda el mejor modelo 
    model = KMeans(n_clusters=elbow_k, random_state=42, n_init='auto').fit(X)
    score = silhouette_score(X, model.labels_)
    best_k, best_model = elbow_k, KMeans(n_clusters=elbow_k, random_state=42, n_init='auto').fit(X)

    # Visualización de resultados
    _plot_metrics(metrics, best_k)
    
    print(f"Número óptimo de clusters: {elbow_k} (score de silueta: {score:.4f}) (score de inercia: {model.inertia_:.4f})")
    return best_k, best_model


def _plot_metrics(metrics: dict, best_k: int) -> None:
    """Función auxiliar para graficar las métricas."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Gráfico de Silhouette Score
    ax1.plot(metrics['k_values'], metrics['silhouette_scores'], 'bo-')
    # ax1.axvline(x=best_k, color='r', linestyle='--')
    ax1.set(xlabel='Número de Clusters', 
           ylabel='Silhouette Score',
           title='Método de la Silueta')
    
    # Gráfico del Método del Codo
    ax2.plot(metrics['k_values'], metrics['inertias'], 'bo-')
    ax2.set(xlabel='Número de Clusters', 
           ylabel='Inercia',
           title='Método del Codo')
    
    plt.tight_layout()
    plt.show()


def build_preprocessor(num_cols):
    """
    Construye el preprocesador para variables numéricas y categóricas.
    
    Args:
        num_cols: lista de columnas numéricas
        cat_cols: lista de columnas categóricas
        
    Returns:
        ColumnTransformer con los transformadores
    """
    numeric_standard = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])
    
    numeric_minmax = Pipeline(steps=[
        ('scaler', MinMaxScaler())
    ])
    
    # Creamos el ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num_std', numeric_standard, num_cols),
            ('num_minmax', numeric_minmax, num_cols),  # Aplicamos ambos escalados a las numéricas
        ],
        remainder='drop'  # Eliminamos columnas no especificadas
    )
    
    return preprocessor


# Función para calcular la distancia de Mahalanobis
def mahalanobis_distance(X):
    # Calcular la matriz de covarianza
    cov_matrix = np.cov(X, rowvar=False)
    # Calcular la inversa de la matriz de covarianza
    inv_cov_matrix = inv(cov_matrix)
    # Calcular la matriz de distancias de Mahalanobis
    distances = pdist(X, lambda u, v: np.sqrt(np.dot(np.dot((u - v).T, inv_cov_matrix), (u - v))))
    return distances


def create_italy_choropleth(
    df,
    color_column='value',
    location_id='geoid',
    hover_vars=None,  # Dictionary of {column: label} for hover data}
    orden_categorias=None, # List of for order data}
    title="Distribución Poblacional en Italia por Grupos Demográficos",
    source_text="Fuente: XYZ",
    center_lat=41.93309,
    center_lon=12.13129,
    zoom=3.8,
    opacity=0.4,
    map_style='carto-positron',
    color_scale=px.colors.sequential.Blues
):
    """Create an elegant choropleth map of Italy with customizable hover data.
    
    Args:
        df: GeoDataFrame with geographical data and values to plot
        color_column: Column name for color values
        location_id: Column name for location IDs (must match geojson properties)
        hover_vars: Dictionary of {column: label} for hover data display
                   Example: {'DEN_PROV': 'Territorio', 'value': 'Unidades'}
        title: Map title
        source_text: Text for data source annotation
        center_lat: Latitude for map center
        center_lon: Longitude for map center
        zoom: Initial zoom level
        opacity: Opacity of choropleth fill
        map_style: Mapbox style specification
        color_scale: Color scale for the choropleth
    
    Returns:
        A Plotly Figure object
    """
    # Set default hover variables if none provided
    if hover_vars is None:
        hover_vars = {
            'DEN_PROV': 'Territorio',
            color_column: 'Valor'
        }
    
    # Prepare hover data and labels
    hover_data = list(hover_vars.keys())
    hover_labels = {col: label for col, label in hover_vars.items()}
    
    fig = px.choropleth_mapbox(
        df,
        geojson=df.__geo_interface__,
        locations=location_id,
        color=color_column,
        category_orders={color_column: orden_categorias},
        hover_data=hover_data,
        labels=hover_labels,
        featureidkey=f'properties.{location_id}',
        center={'lat': center_lat, 'lon': center_lon},
        mapbox_style=map_style,
        zoom=zoom,
        opacity=opacity,
        title=title,
        # color_continuous_scale=color_scale
    )

    # Generate dynamic hover template
    hover_template_parts = [
        f"<b>{label}</b>: %{{customdata[{i}]}}"
        if not isinstance(df[col].iloc[0], (int, np.int64, float))
        else (
            f"<b>{label}</b>: %{{customdata[{i}]:,}}"
            if df[col].iloc[0] > 1
            # else f"<b>{label}</b>: %{{customdata[{i}]:.3f}}"
            else f"<b>{label}</b>: %{{customdata[{i}]:.2%}}"
        )
        for i, (col, label) in enumerate(hover_vars.items())
    ]

    fig.update_traces(
        hovertemplate="<br>".join(hover_template_parts) + "<extra></extra>",
        marker_line_width=0.5,
        marker_line_color='white'
    )

    # Update layout for better appearance
    fig.update_layout(
        margin={'l': 0, 't': 60, 'r': 0, 'b': 40},
        title={
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 16}
        },
        annotations=[dict(
            text=source_text,
            x=0.5,
            y=-0.1,
            showarrow=False,
            xref="paper",
            yref="paper",
            font={'size': 10}
        )],
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial"
        )
    )

    # Update colorbar
    colorbar_title = hover_vars.get(color_column, 'Value')
    fig.update_coloraxes(
        colorbar=dict(
            title=dict(text=colorbar_title, side='right'),
            lenmode="fraction",
            len=0.6,
            yanchor="middle",
            y=0.5,
            x=0,
            thickness=15,
            tickformat=","
        )
    )

    return fig

In [ ]:
geo_provincias = cargar_datos_geo('provincias')

regiones = cargar_datos('regiones')

censo = cargar_datos('censo_tranformado')
censo = censo[censo['marital_status'].isin(['never married', 'married', 'divorced'])]

censo['grupo_edad'] = 'adultos' 
censo.loc[censo['age'] < 18, 'grupo_edad'] = 'jóvenes'
censo.loc[censo['age'] > 65, 'grupo_edad'] = 'mayores'

df = pd.merge(censo, regiones, on=['itter107'])

df = calcular_proporciones(df, code='COD_REG')

df.head()

In [ ]:
# Construir el preprocesador
preprocessor = build_preprocessor(numeric_features)

# Aplicar preprocesamiento
X_processed = preprocessor.fit_transform(df)

# Encontrar el K óptimo
best_k, best_model = optimal_kmeans(X_processed, max_k=10)

In [ ]:
df['group'] = best_model.predict(X_processed)
df['group'] = df['group'].astype(str)
df.head()

In [ ]:
geo_regiones = cargar_datos_geo('regiones')
df = pd.merge(geo_regiones, df, on=['COD_REG'])
df.head()

In [ ]:
df['group'] = df['group'].astype(str)
df.info()

In [ ]:
mapita = create_italy_choropleth(
    df,
    color_column='group',
    hover_vars={'territory':'Territorio',
                'group':'Grupo',
                'value':'Unidades'
                },
    orden_categorias=orden_categorias,
    # value_label='Unidades',
    location_id='COD_REG',
    title="Distribución Poblacional en Italia por Grupos",
    source_text="Fuente: Datos demográficos 2025 | Visualización: Python",
    # center_lat=41.93309,
    # center_lon=12.13129,
    # zoom=3.8,
    # opacity=0.4,
    # map_style='carto-positron'
)
# mapita.write_html(outputs / 'distribucion_poblacional_grupos_italia.html')
mapita

In [ ]:
provincias = cargar_datos('provincias')

censo = cargar_datos('censo_tranformado')
censo = censo[censo['marital_status'].isin(['never married', 'married', 'divorced'])]

censo['grupo_edad'] = 'adultos' 
censo.loc[censo['age'] < 18, 'grupo_edad'] = 'jóvenes'
censo.loc[censo['age'] > 65, 'grupo_edad'] = 'mayores'

df = pd.merge(censo, provincias, on=['itter107'])

df = calcular_proporciones(df, code='COD_PROV')

df.head()

In [ ]:
# Construir el preprocesador
preprocessor = build_preprocessor(numeric_features)

# Aplicar preprocesamiento
X_processed = preprocessor.fit_transform(df)

# Encontrar el K óptimo
best_k, best_model = optimal_kmeans(X_processed, max_k=10)

In [ ]:
df['group'] = best_model.predict(X_processed)
df['group'] = df['group'].astype(str)
df = pd.merge(geo_provincias, df, on=['COD_PROV'])
df.head()

In [ ]:
mapita = create_italy_choropleth(
    df,
    color_column='group',
    hover_vars={"territory": 'Territorio',
                'group': 'Grupo',
                'value':'Unidades', 
                "females":'Mujeres', 
                "jóvenes":'Jóvenes', 
                "married":'Casados'},
    orden_categorias=orden_categorias,          
    # value_label='Unidades',
    location_id='COD_PROV',
    title="Distribución Poblacional en Italia por Grupos",
    source_text="Fuente: Datos demográficos 2025 | Visualización: Python",
    # center_lat=41.93309,
    # center_lon=12.13129,
    # zoom=3.8,
    # opacity=0.4,
    # map_style='carto-positron'
)
mapita.write_html(outputs / 'distribucion_poblacional_grupos_italia.html')
mapita

In [ ]:
# Crear el mapa
fig, ax = plt.subplots(figsize=(9, 6), facecolor='#f5f5f5')

# Graficar los polígonos con parámetros mejorados
df.plot(ax=ax, 
        column='group', 
        legend=True, 
        cmap='viridis', 
        edgecolor='white', 
        linewidth=0.4,
        alpha=0.9,  # Ligera transparencia
        missing_kwds={'color': 'lightgray'})  # Color para datos faltantes

# Mejorar el título y anotaciones
ax.set_title('Distribución Poblacional en Italia por Grupos Demográficos\n', 
             fontsize=18, 
             pad=20, 
             fontweight='bold',
             color='#333333')

# Mejorar la leyenda
legend = ax.get_legend()
if legend:
    legend.set_title('Grupos', prop={'size': 12, 'weight': 'bold'})
    for text in legend.get_texts():
        text.set_fontsize(10)

# Añadir créditos o fuente
ax.annotate('Fuente: Datos demográficos 2025 | Visualización: Python', 
            xy=(0.5, 0.02), 
            xycoords='figure fraction',
            ha='center', 
            fontsize=10, 
            color='gray')

# Ajustar bordes y fondo
ax.set_axis_off()
fig.set_facecolor('#f5f5f5')

# Guardar en alta calidad
plt.savefig(outputs/'distribucion_poblacional_grupos_italia.png', 
           dpi=300, 
           bbox_inches='tight', 
           facecolor=fig.get_facecolor())

# Mostrar el mapa
plt.tight_layout()
plt.show()

In [ ]:
df_s = df.copy()
df_s = df_s.loc[:,['territory', 'value', 'females',
       'males', 'adultos', 'jóvenes', 'mayores', 'divorced', 'married',
       'never married', 'group']]
df_s = df_s.groupby('group')[['value', 'females',
       'males', 'adultos', 'jóvenes', 'mayores', 'divorced', 'married',
       'never married']].agg(['mean', 'std', 'min', 'max'])
df_s.columns = ['_'.join(col).strip() for col in df_s.columns.values]
df_s = df_s.round(4)
df_s

## Resumen de los Grupos del Censo de Italia

### Grupo 0: "Provincias con Población Envejecida y Alta Tasa de Divorcios"

**Particularidad**: Este grupo tiene la proporción más alta de personas mayores (21.27%) y la tasa más alta de divorcios (5.46%). Además, presenta la menor proporción de jóvenes (14.16%).

Detalles:
- Población promedio: 272,848 personas.
- Proporción de mujeres: 48.41%, ligeramente menor que otros grupos.
- Estado civil: 48.32% casados, 46.22% solteros.

### Grupo 1: "Provincias con Población Equilibrada y Matrimonios Estables"

**Particularidad**: Este grupo destaca por tener la proporción más alta de casados (51.94%) y la menor tasa de divorcios (2.53%). La distribución por género es casi equilibrada (47.99% mujeres, 52.01% hombres).

Detalles:
- Población promedio: 322,130 personas.
- Edades: 65.87% adultos, 16.19% jóvenes, 17.94% mayores.
- Estado civil: 51.94% casados, 45.52% solteros.

### Grupo 2: "Provincias con Diversidad en Estado Civil y Población Joven"

**Particularidad**: Este grupo tiene una tasa de divorcios intermedia (4.39%) y una proporción significativa de jóvenes (15.60%). Además, presenta una alta variabilidad en el estado civil.

Detalles:
- Población promedio: 312,698 personas.
- Proporción de mujeres: 48.07%.
- Estado civil: 48.85% casados, 46.77% solteros.

### Grupo 3: "Provincias con Población Urbana y Alta Movilidad Social"

**Particularidad**: Este grupo es el más pequeño (solo 3 provincias) y tiene la población más grande (3,245,079 personas en promedio). Destaca por tener la proporción más alta de mujeres (49.34%) y una alta tasa de solteros (49.35%).

Detalles:
- Edades: 66.50% adultos, 17.30% jóvenes, 16.20% mayores.
- Estado civil: 46.93% casados, 49.35% solteros.

### Grupo 4: "Provincias con Población Prospera y Estructura Familiar Tradicional"

**Particularidad**: Este grupo tiene la segunda población más grande (646,403 personas en promedio) y una proporción equilibrada de género (48.90% mujeres). Además, presenta una alta proporción de casados (51.93%) y baja tasa de divorcios (2.55%).

Detalles:
- Edades: 65.83% adultos, 16.86% jóvenes, 17.31% mayores.
- Estado civil: 51.93% casados, 45.53% solteros.

### Grupo 5: "Provincias con Alta Diversidad Demográfica y Población en Transición"

**Particularidad**: Este grupo tiene la mayor variabilidad en tamaño de población (desde 113,335 hasta 2,029,234 personas) y una tasa de divorcios relativamente alta (4.65%). Además, presenta una proporción equilibrada de solteros (48.30%) y casados (47.05%).

Detalles:
- Proporción de mujeres: 48.49%.
- Edades: 65.53% adultos, 16.64% jóvenes, 17.83% mayores.
- Estado civil: 47.05% casados, 48.30% solteros.

**Conclusión:**

Cada grupo refleja características demográficas y sociales únicas, desde provincias con poblaciones envejecidas y altas tasas de divorcios (Grupo 0) hasta aquellas con estructuras familiares tradicionales y poblaciones prósperas (Grupo 4). El análisis permite identificar patrones clave para políticas públicas o estudios sociodemográficos más detallados.

## Análisis Complementario del Gráfico

El gráfico de distribución poblacional en Italia refleja las características clave de los grupos analizados, destacando:

**Grupos con Mayor Población:** Grupo 3 (urbano) y Grupo 5 (diverso) tienen las poblaciones más grandes, con promedios de 3.2 millones y 657,549 personas, respectivamente. Esto sugiere que corresponden a áreas metropolitanas o provincias con alta densidad.

**Equilibrio de Género:** Todos los grupos muestran una proporción cercana al 48-49% mujeres vs. 51-52% hombres, con ligeras variaciones. Grupo 3 tiene la mayor proporción de mujeres (49.34%), mientras que Grupo 1 tiene la menor (47.99%).

**Estructura Etaria:** Grupo 0 destaca por su población envejecida (21.3% mayores), mientras que Grupo 5 tiene la mayor proporción de jóvenes (16.6%). Grupos 1 y 4 muestran estructuras similares, con altos porcentajes de adultos (~65-66%) y bajas tasas de jóvenes/mayores, indicando estabilidad demográfica.

**Tendencias en Estado Civil**: Grupos 1 y 4 tienen las tasas más altas de matrimonio (~52%) y las más bajas de divorcio (~2.5%), asociadas a estructuras familiares tradicionales. Grupo 0 y Grupo 5 presentan mayor diversidad en estado civil, con tasas altas de solteros (46-48%) y divorcios (4.6-5.5%).

## Conclusiones Visuales del Gráfico

**Disparidades Geográficas:** Las provincias urbanas (Grupo 3) contrastan con las rurales o envejecidas (Grupo 0).

**Transiciones Demográficas:** Grupos como el 5 reflejan cambios sociales (ej.: más solteros y diversidad en tamaño poblacional).

**Políticas Públicas:** El gráfico podría ayudar a identificar necesidades específicas, como atención a adultos mayores (Grupo 0) o programas para jóvenes (Grupo 5).

**Nota:** Si el gráfico incluye mapas o pirámides poblacionales, estos detalles reforzarían las diferencias espaciales y generacionales.

In [ ]:
df_h = df.copy()
df_h = df_h.loc[:,['value', 'females', 'adultos', 'jóvenes', 'divorced', 'married']]

# Calcular la matriz de distancias de Mahalanobis
dist_matrix = mahalanobis_distance(df_h)

# Aplicar clustering jerárquico
Z = linkage(dist_matrix, method='ward')

# Visualizar el dendrograma
plt.figure(figsize=(10, 5))
dendrogram(Z)
plt.title("Dendrograma con Distancia de Mahalanobis")
plt.xlabel("Índices de los puntos")
plt.ylabel("Distancia de Mahalanobis")
plt.show()

In [ ]:
umbral = 7
df['cluster'] = fcluster(Z, t=umbral, criterion='distance')
df['cluster'] = df['cluster'].astype(str)
df.head()

In [ ]:
mapita = create_italy_choropleth(
    df,
    color_column='cluster',
    hover_vars={"territory": 'Territorio',
                'cluster': 'Grupo',
                'value':'Unidades', 
                "females":'Mujeres', 
                "jóvenes":'Jóvenes', 
                "married":'Casados'},
    orden_categorias=orden_categorias,         
    # value_label='Unidades',
    location_id='COD_PROV',
    title="Distribución Poblacional en Italia por Grupos",
    source_text="Fuente: Datos demográficos 2025 | Visualización: Python",
    # center_lat=41.93309,
    # center_lon=12.13129,
    # zoom=3.8,
    # opacity=0.4,
    # map_style='carto-positron'
)
# mapita.write_html(outputs / 'distribucion_poblacional_grupos_italia2.html')
mapita

In [ ]:
# Crear el mapa
fig, ax = plt.subplots(figsize=(9, 6), facecolor='#f5f5f5')

# Graficar los polígonos con parámetros mejorados
df.plot(ax=ax, 
        column='cluster', 
        legend=True, 
        cmap='viridis', 
        edgecolor='white', 
        linewidth=0.4,
        alpha=0.9,  # Ligera transparencia
        missing_kwds={'color': 'lightgray'})  # Color para datos faltantes

# Mejorar el título y anotaciones
ax.set_title('Distribución Poblacional en Italia por Grupos Demográficos\n', 
             fontsize=18, 
             pad=20, 
             fontweight='bold',
             color='#333333')

# Mejorar la leyenda
legend = ax.get_legend()
if legend:
    legend.set_title('Grupos', prop={'size': 12, 'weight': 'bold'})
    for text in legend.get_texts():
        text.set_fontsize(10)

# Añadir créditos o fuente
ax.annotate('Fuente: Datos demográficos 2025 | Visualización: Python', 
            xy=(0.5, 0.02), 
            xycoords='figure fraction',
            ha='center', 
            fontsize=10, 
            color='gray')

# Ajustar bordes y fondo
ax.set_axis_off()
fig.set_facecolor('#f5f5f5')

# Guardar en alta calidad
plt.savefig(outputs/'distribucion_poblacional_grupos_italia2.png', 
           dpi=300, 
           bbox_inches='tight', 
           facecolor=fig.get_facecolor())

# Mostrar el mapa
plt.tight_layout()
plt.show()

In [ ]:
df= df.loc[:,['COD_PROV', 'itter107', 'territory', 'group', 'cluster']]
cargar_en_db(df, 'groups')